In [2]:
import pandas as pd
from IPython.display import clear_output

#labels = pd.read_excel('/content/Training Data labels.xlsx')
data = pd.read_csv('/content/Retail_Customer.csv')

data['Visit_Date'] = pd.to_datetime(data['Visit_Date'])

data = data.sort_values(by='Visit_Date')

grouped_data = data.groupby('CustomerID')

features = pd.DataFrame(columns=[
  'CustomerID',
  'Total_Revenue',
  'Max_Purchase_In_A_Day',
  'Min_Purchase_In_A_Day',
  'Total_Visit_Days',
  'Standard_Deviation_In_Sales',
  'W1_Total_Sales',
  'W1_Visit_Days',
  'W2_Total_Sales',
  'W2_Visit_Days',
  'W3_Total_Sales',
  'W3_Visit_Days',
  'W4_Total_Sales',
  'W4_Visit_Days',
  'W5_Total_Sales',
  'W5_Visit_Days',
  'Friday_Visit_Days',
  'Friday_Total_Sales',
  'Thursday_Visit_Days',
  'Thursday_Total_Sales',
  'Tuesday_Visit_Days',
  'Tuesday_Total_Sales',
  'Saturday_Visit_Days',
  'Saturday_Total_Sales',
  'Wednesday_Visit_Days',
  'Wednesday_Total_Sales',
  'Sunday_Visit_Days',
  'Sunday_Total_Sales',
  'Monday_Visit_Days',
  'Monday_Total_Sales',
  'Favourite_Visit_Day_Revenue_Wise',
  'Favourite_Visit_Day_Vists_Wise',
  'D4_Visit_Flag',
  'D4_Sales_Amount',
  'D0_Visit_Flag',
  'D0_Sales_Amount',
  'D2_Visit_Flag',
  'D2_Sales_Amount',
  'D5_Visit_Flag',
  'D3_Visit_Flag',
  'D3_Sales_Amount',
  'D1_Visit_Flag',
  'D1_Sales_Amount',
  'Days_Since_Last_Visit',
  'Label'
])

index = 0
reference_date = pd.Timestamp('2014-10-19')
num_weeks = 5

for customer_id, customer_data in grouped_data:
  clear_output(wait=True)
  print("Index:", index)

  features.loc[index, 'CustomerID'] = customer_id
  features.loc[index, 'Total_Revenue'] = customer_data['Total_Purchases_In_USD'].sum()
  features.loc[index, 'Max_Purchase_In_A_Day'] = customer_data['Total_Purchases_In_USD'].max()
  features.loc[index, 'Min_Purchase_In_A_Day'] = customer_data['Total_Purchases_In_USD'].min()
  features.loc[index, 'Total_Visit_Days'] = len(customer_data['Visit_Date'].dt.date.unique())
  features.loc[index, 'Standard_Deviation_In_Sales'] = customer_data['Total_Purchases_In_USD'].std()

  for i in range(num_weeks):
    end_date = reference_date - pd.Timedelta(weeks=i)
    start_date = end_date - pd.Timedelta(days=6)

    week_data = customer_data[(customer_data['Visit_Date'] >= start_date) & (customer_data['Visit_Date'] <= end_date)]

    if not week_data.empty:
      features.loc[index, f'W{i+1}_Total_Sales'] = week_data['Total_Purchases_In_USD'].sum()
      features.loc[index, f'W{i+1}_Visit_Days'] = len(week_data['Visit_Date'].dt.date.unique())

  day_wise_sales = customer_data.groupby(customer_data['Visit_Date'].dt.day_name())['Total_Purchases_In_USD'].sum()
  day_wise_visit_days = customer_data.groupby(customer_data['Visit_Date'].dt.day_name())['Visit_Date'].nunique()

  for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
    features.loc[index, f'{day}_Total_Sales'] = day_wise_sales.get(day, 0)
    features.loc[index, f'{day}_Visit_Days'] = day_wise_visit_days.get(day, 0)

  features.loc[index, 'Favourite_Visit_Day_Revenue_Wise'] = day_wise_sales.idxmax()
  features.loc[index, 'Favourite_Visit_Day_Vists_Wise'] = day_wise_visit_days.idxmax()

  features.loc[index, 'D4_Visit_Flag'] = int(any((reference_date - pd.Timedelta(days=4)) == customer_data['Visit_Date']))
  features.loc[index, 'D4_Sales_Amount'] = customer_data[customer_data['Visit_Date'] == (reference_date - pd.Timedelta(days=4))]['Total_Purchases_In_USD'].sum()

  features.loc[index, 'D0_Visit_Flag'] = int(any(reference_date == customer_data['Visit_Date']))
  features.loc[index, 'D0_Sales_Amount'] = customer_data[customer_data['Visit_Date'] == reference_date]['Total_Purchases_In_USD'].sum()

  features.loc[index, 'D2_Visit_Flag'] = int(any((reference_date - pd.Timedelta(days=2)) == customer_data['Visit_Date']))
  features.loc[index, 'D2_Sales_Amount'] = customer_data[customer_data['Visit_Date'] == (reference_date - pd.Timedelta(days=2))]['Total_Purchases_In_USD'].sum()

  features.loc[index, 'D5_Visit_Flag'] = int(any((reference_date - pd.Timedelta(days=5)) == customer_data['Visit_Date']))

  features.loc[index, 'D3_Visit_Flag'] = int(any((reference_date - pd.Timedelta(days=3)) == customer_data['Visit_Date']))
  features.loc[index, 'D3_Sales_Amount'] = customer_data[customer_data['Visit_Date'] == (reference_date - pd.Timedelta(days=3))]['Total_Purchases_In_USD'].sum()

  features.loc[index, 'D1_Visit_Flag'] = int(any((reference_date - pd.Timedelta(days=1)) == customer_data['Visit_Date']))
  features.loc[index, 'D1_Sales_Amount'] = customer_data[customer_data['Visit_Date'] == (reference_date - pd.Timedelta(days=1))]['Total_Purchases_In_USD'].sum()

  last_visit_date = customer_data['Visit_Date'].max()
  if pd.notnull(last_visit_date):
    days_since_last_visit = (reference_date - last_visit_date).days
    features.loc[index, 'Days_Since_Last_Visit'] = days_since_last_visit
  else:
    features.loc[index, 'Days_Since_Last_Visit'] = None

  if ((reference_date - last_visit_date).days <=7):
    features.loc[index, 'Label'] = "Not Churned"
  else:
    features.loc[index, 'Label'] = "Churned"

  index += 1

# Save the features to a new CSV file
features.to_csv('customer_features.csv', index=False)


Index: 86340
